In [ ]:
from bs4 import BeautifulSoup
import json
import re

with open("../output/구강_구개열_crop_parsing.json", "r", encoding="utf-8") as f:
    data = json.load(f)

html = data['content']['html']
soup = BeautifulSoup(html, 'html.parser')

# 병명 추출 (h1 없을 때 대비)
disease_tag = soup.find("h1")

if disease_tag is None:
    print("[⚠️경고] h1 태그를 찾을 수 없습니다.")
    
    # h1이 없으면 font-size:22px인 첫 <p>나 <header>를 찾아서 병명 추출
    for tag in soup.find_all(["p", "header"]):
        style = tag.get("style", "")
        if "font-size:22px" in style:
            disease_tag = tag
            print("[⚠️] fontsize 22 태그로 추출했습니다.")

            break

# 병명 최종 추출
if disease_tag:
  disease_name = disease_tag.get_text(strip=True)
else:
  disease_name = "Unknown Disease"
  print("[⚠️경고] 병명 추출 실패: h1 또는 font-size:22px 태그를 찾을 수 없습니다.")

# 3. 그래도 없으면 '• 콘텐츠명 :' 포함된 텍스트에서 추출
if not disease_name:
    for tag in soup.find_all("p"):
        text = tag.get_text(strip=True)
        if "• 콘텐츠명" in text:
            # "• 콘텐츠명 : 구개열" 형식에서 오른쪽 값만 추출
            match = re.search(r"콘텐츠명\s*:\s*(.+)", text)
            if match:
                disease_name = match.group(1).strip()
                break
            
print(f'✅병명: {disease_name}')


✅병명: 구개열


In [43]:
chunks = []
current_section = None
current_text = []

for tag in soup.find_all(["h1", "h2", "header", "p"]):
    text = tag.get_text(strip=True)
    style = tag.get("style", "")

    if not text:
        continue

    # font-size 정보 추출
    font_size = 0
    if "font-size:" in style:
        try:
            font_size = int(style.split("font-size:")[1].replace("px", "").strip())
        except:
            pass

    # 제목 판단: tag 자체가 heading이거나, font-size가 큰 <p> 태그
    is_heading = (
        tag.name in ["h1", "h2", "header"] or font_size >= 18
    )

    if is_heading:
        # 이전 섹션 저장
        if current_section and current_text:
            chunks.append({
                "disease": disease_name,
                "section": current_section,
                "content": "\n".join(current_text)
            })
            current_text = []

        current_section = text
    else:
        current_text.append(text)

# 마지막 섹션 저장
if current_section and current_text:
    chunks.append({
        "disease": disease_name,
        "section": current_section,
        "content": "\n".join(current_text)
    })


In [44]:
print("done!")

# 결과를 JSON 파일로 저장 (가독성 있게)
with open("구개열_crop_test.json", "w", encoding="utf-8") as f_json:
    json.dump(chunks, f_json, ensure_ascii=False, indent=2)

# 또는 텍스트 파일로 보기 좋게 저장
with open("구개열_crop_test.txt", "w", encoding="utf-8") as f_txt:
    for chunk in chunks:
        f_txt.write(f"[{chunk['section']}]\n")
        f_txt.write(chunk["content"] + "\n\n")


done!
